### Cloud Persistent Disks and Local SSDs<br>

***

<br>

**Block Storage**<br>

by default, each Compute Engine instances have a single boot persistent disks that contains OS<br>
you can add more persistent disks or SSD to your instance<br>

- **Persistent Disk (pd)**<br>

Option 1: Zonal - Regional<br>
mixed with<br>
Option 2: Standard - Balanced - SSD<br>

Persistent disks are durable network storage devices, that your instances can access.<br>
They're not physically attached disks, but network disks that are connected in Google's internal network<br>

Persistent disks are independent, it can persist after instances has been terminated<br>
you can also detach the disk and move it to other instances if you need to<br>

disk resize feature allows you to rescale, with no downtime and add additional disks<br>
They're encrypted by default<br>

pd can be up to 64TB in size<br>
most instances can have up to 128 persistent disks and 257TB total persistent disks space attached<br>
Shared core machine types are limited to 16 persistent disks and 3TB total persistent disk space<br>

Option 1 is picking the geographic options.<br>
Zonal - disks that are only available in 1 zone, 1 region<br>
most commonly used disks for day to day usage<br>
they can't survive an outage of that zone, and can be affected by data loss<br>
this is where snapshot comes into play<br>
Zonal disks can work with any machine types, including pre-defined, shared-core, custom machine type<br>

Regional - similar qualities to zonal, but<br>
regional provides durable storage, replication of data in 2 zones of same region<br>
if you're designing a system that uses high compute engine, you need to use this<br>
also designed to work with regional managed instance groups<br>

regional disks are slower than zonal disks<br>
so consider it if write performance is less critical and data availability across the zones<br>
regional standard disks have 200GB size minimum requirement<br>
and regional disks can't work with memory/compute optimized machine types<br>
<br>

**pd-standard**<br>

backed by standard hard disk drives(HDD)<br>
large data processing workloads that primarily use sequential I/Os<br>
lowest pricd persistent disks<br>
<br>

**pd-balanced**<br>

alternative to SSD persistent disks, balanced performance and cost<br>
same maximum IOPS and ssd-pd, lower IOPS per GB<br>
general purpose<br>
price in between standard and SSD persistent disks<br>

about 3x faster than pd-standard in performance at max IOPS<br>
<br>

**pd-ssd**<br>

fastests persistent disks<br>
enterprise applications and high-performance databases that demand lower latency and more IOPS<br>
single-digit ms latencies<br>
highest priced persistent disks<br>

about 5x faster than pd-balanced in max IOPS<br>
<br>

- **Local SSD**<br>

Option 1: SCSI - NVME<br>
Option 2: Local SSD<br>

physically attached to your server that hosts local instances<br>
local ssds have higher throughput/lower latency than pd<br>
it's because it's physically attached and data doesn't have to travel over the network<br>

data persists until instance is stopped or deleted<br>
each local ssd 375GB, attach max of 24 local ssd for total of 9TB per instance<br>
fast scratch disk or cache<br>

SCSI - older protocol, made for harddrives and hold limitation of 1 queues per command<br>
Non violatile memory express (NVME) - is a newer protocol, use flash memory, designed upto 64k queues<br>
only available for n1,n2 and compute optimized machine types<br>

about 10-25x faster than pd-ssd in max IOPS<br>
<br>

performance scaling in block storage in compute engine<br>

pd performance scales with the size of disk, and number of vCPUs<br>
pd performance scales linearly until the limits of volume or limits of each compute instance<br>
<br>

**Demo**<br>

this demo is on how to manage and interface with disks in Compute Engine<br>

**creating an instance**<br>

check VPC network for default network<br>
and check the firewall rules for ssh<br>

go to the Compute Engine and create a new instance<br>
name: instance1<br>
region: us-east1, us-east1-b<br>
machine type: e2 micro<br>
service account: Set access for each API: Compute Engine: Read/Write<br>
<br>

**create a separate persistent disks**<br>

Compute Engine -> Disks<br>
disks for the instance is there, with 10GB<br>
but, click on create a new disk<br>
name: newpd<br>
region: us-east1, us-east1-b<br>
(if you click replicate this disk, you change it to zonal to regional pd)<br>
source type: blank disk: size 100GB (look at the est. performance)<br>
label: key: env, value: testing<br>
<br>

**attach to the disk**<br>

back to the VM instance and ssh into the instance<br>

> lsblk<br>
> gcloud compute instances attach-disk instance1 --disk newpd --zone us-east1-b<br>
> lsblk //check if it's been attached<br>

this is formatting the disk to ext4 format<br>

> sudo file -s /dev/sdb<br>
> sudo mkfs.ext4 -F /dev/sdb<br>
> sudo file -s /dev/sdb //check if it's been formatted<br>

mount the disk<br>

> sudo mkdir /newpd<br>
> sudo mount /dev/sdb /newpd<br>
> lsblk<br>

sdb has been mounted as a /newpd<br>
now we can interact with the disk<br>

> cd /newpd/<br>
> ls<br>

lost+found directory is found in every linux system<br>
to place orphan or corrupted files to place here<br>

> sudo nano fileoftext.txt<br>

type in the text and press Ctrl+O to save<br>
Hit enter to verify and hit Ctrl+X to exit<br>

> ls<br>
> df -k<br>
> sudo reboot<br>

wait for a minute, before clicking SSH again<br>

> lsblk<br>

newpd didn't sustain through reboot, due to a config file in linux<br>
that points to which partitions mount automatically on startup<br>

> sudo blkid /dev/sdb<br>

copy the UUID<br>

> sudo nano /etc/fstab<br>

find UUID for another partition<br>
move down the line and add a new line<br>

> UUID=[paste the UUID that I just copied] /newpd ext4 defaults,nofail<br>

Ctrl+O to save, Enter to verify, Ctrl+X to exit<br>

> sudo mount -a<br>
> lsblk<br>

this is a common task working on linux attaching a disk, and also can be scripted<br>

**then resize the disk**<br>

100GB -> 150GB<br>
go back to disks, click edit<br>
150GB and click save<br>

in terminal to resize the disk<br>
> gcloud compute disks resize newpd --size 150 --zone us-east1-b<br>

Remember: you can only make it bigger, NEVER smaller<br>

> df -k<br>

only displays 100GB, because we didn't allocate the raw blocks to the file systems<br>

> sudo resize2fs /dev/sdb<br>
> df -k<br>

> ls -al<br>
> ls<br>

<br>

**deleting the disk and clean up**<br>

detach the disk<br>
> gcloud compute instances detach-disk instance1 --disk newpd --zone us-east1-b<br>

head back to the console and delete the disk<br>
<br>